#### Copyright 2019 The TensorFlow Authors.

In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Classification on imbalanced data

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/structured_data/imbalanced_data"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/structured_data/imbalanced_data.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/structured_data/imbalanced_data.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/tutorials/structured_data/imbalanced_data.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

This tutorial demonstrates how to classify a highly imbalanced dataset in which the number of examples in one class greatly outnumbers the examples in another. You will work with the [Credit Card Fraud Detection](https://www.kaggle.com/mlg-ulb/creditcardfraud) dataset hosted on Kaggle. The aim is to detect a mere 492 fraudulent transactions from 284,807 transactions in total. You will use [Keras](https://www.tensorflow.org/guide/keras/overview) to define the model and [class weights](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/Model) to help the model learn from the imbalanced data. .

This tutorial contains complete code to:

* Load a CSV file using Pandas.
* Create train, validation, and test sets.
* Define and train a model using Keras (including setting class weights).
* Evaluate the model using various metrics (including precision and recall).
* Select a threshold for a probabilistic classifier to get a deterministic classifier.
* Try and compare with class weighted modelling and oversampling.

## Setup

In [2]:
import tensorflow as tf
from tensorflow import keras

import os
import tempfile

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

import sklearn
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [3]:
mpl.rcParams['figure.figsize'] = (12, 10)
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

## Data processing and exploration

### Download the Kaggle Credit Card Fraud data set

Pandas is a Python library with many helpful utilities for loading and working with structured data. It can be used to download CSVs into a Pandas [DataFrame](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html#pandas.DataFrame).

Note: This dataset has been collected and analysed during a research collaboration of Worldline and the [Machine Learning Group](http://mlg.ulb.ac.be) of ULB (Université Libre de Bruxelles) on big data mining and fraud detection. More details on current and past projects on related topics are available [here](https://www.researchgate.net/project/Fraud-detection-5) and the page of the [DefeatFraud](https://mlg.ulb.ac.be/wordpress/portfolio_page/defeatfraud-assessment-and-validation-of-deep-feature-engineering-and-learning-solutions-for-fraud-detection/) project

In [4]:
file = tf.keras.utils
raw_df = pd.read_csv('https://storage.googleapis.com/download.tensorflow.org/data/creditcard.csv')
raw_df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [5]:
raw_df[['Time', 'V1', 'V2', 'V3', 'V4', 'V5', 'V26', 'V27', 'V28', 'Amount', 'Class']].describe()

,Time,V1,V2,V3,V4,V5,V26,V27,V28,Amount,Class
count,284807.000000,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,284807.000000,284807.000000
mean,94813.859575,3.918649e-15,5.682686e-16,-8.761736e-15,2.811118e-15,-1.552103e-15,1.701640e-15,-3.662252e-16,-1.217809e-16,88.349619,0.001727
std,47488.145955,1.958696e+00,1.651309e+00,1.516255e+00,1.415869e+00,1.380247e+00,4.822270e-01,4.036325e-01,3.300833e-01,250.120109,0.041527
min,0.000000,-5.640751e+01,-7.271573e+01,-4.832559e+01,-5.683171e+00,-1.137433e+02,-2.604551e+00,-2.256568e+01,-1.543008e+01,0.000000,0.000000
25%,54201.500000,-9.203734e-01,-5.985499e-01,-8.903648e-01,-8.486401e-01,-6.915971e-01,-3.269839e-01,-7.083953e-02,-5.295979e-02,5.600000,0.000000
50%,84692.000000,1.810880e-02,6.548556e-02,1.798463e-01,-1.984653e-02,-5.433583e-02,-5.213911e-02,1.342146e-03,1.124383e-02,22.000000,0.000000
75%,139320.500000,1.315642e+00,8.037239e-01,1.027196e+00,7.433413e-01,6.119264e-01,2.409522e-01,9.104512e-02,7.827995e-02,77.165000,0.000000
max,172792.000000,2.454930e+00,2.205773e+01,9.382558e+00,1.687534e+01,3.480167e+01,3.517346e+00,3.161220e+01,3.384781e+01,25691.160000,1.000000


### Examine the class label imbalance

Let's look at the dataset imbalance:

In [6]:
neg, pos = np.bincount(raw_df['Class'])
total = neg + pos
print('Examples:\n    Total: {}\n    Positive: {} ({:.2f}% of total)\n'.format(
    total, pos, 100 * pos / total))

Examples:
    Total: 284807
    Positive: 492 (0.17% of total)



This shows the small fraction of positive samples.

### Clean, split and normalize the data

The raw data has a few issues. First the `Time` and `Amount` columns are too variable to use directly. Drop the `Time` column (since it's not clear what it means) and take the log of the `Amount` column to reduce its range.

In [7]:
cleaned_df = raw_df.copy()

# You don't want the `Time` column.
cleaned_df.pop('Time')

# The `Amount` column covers a huge range. Convert to log-space.
eps = 0.001 # 0 => 0.1¢
cleaned_df['Log Amount'] = np.log(cleaned_df.pop('Amount')+eps)

Split the dataset into train, validation, and test sets. The validation set is used during the model fitting to evaluate the loss and any metrics, however the model is not fit with this data. The test set is completely unused during the training phase and is only used at the end to evaluate how well the model generalizes to new data. This is especially important with imbalanced datasets where [overfitting](https://developers.google.com/machine-learning/crash-course/generalization/peril-of-overfitting) is a significant concern from the lack of training data.

In [8]:
# Use a utility from sklearn to split and shuffle your dataset.
train_df, test_df = train_test_split(cleaned_df, test_size=0.2)
train_df, val_df = train_test_split(train_df, test_size=0.2)

# Form np arrays of labels and features.
train_labels = np.array(train_df.pop('Class'))
bool_train_labels = train_labels != 0
val_labels = np.array(val_df.pop('Class'))
test_labels = np.array(test_df.pop('Class'))

train_features = np.array(train_df)
val_features = np.array(val_df)
test_features = np.array(test_df)

We check whether the distribution of the classes in the three sets is about the same or not.

In [9]:
print(f'Average class probability in training set:   {train_labels.mean():.4f}')
print(f'Average class probability in validation set: {val_labels.mean():.4f}')
print(f'Average class probability in test set:       {test_labels.mean():.4f}')

Average class probability in training set:   0.0018
Average class probability in validation set: 0.0014
Average class probability in test set:       0.0017


Given the small number of positive labels, this seems about right.

Normalize the input features using the sklearn StandardScaler.
This will set the mean to 0 and standard deviation to 1.

Note: The `StandardScaler` is only fit using the `train_features` to be sure the model is not peeking at the validation or test sets. 

In [10]:
scaler = StandardScaler()
train_features = scaler.fit_transform(train_features)

val_features = scaler.transform(val_features)
test_features = scaler.transform(test_features)

train_features = np.clip(train_features, -5, 5)
val_features = np.clip(val_features, -5, 5)
test_features = np.clip(test_features, -5, 5)


print('Training labels shape:', train_labels.shape)
print('Validation labels shape:', val_labels.shape)
print('Test labels shape:', test_labels.shape)

print('Training features shape:', train_features.shape)
print('Validation features shape:', val_features.shape)
print('Test features shape:', test_features.shape)


Training labels shape: (182276,)
Validation labels shape: (45569,)
Test labels shape: (56962,)
Training features shape: (182276, 29)
Validation features shape: (45569, 29)
Test features shape: (56962, 29)


Caution: If you want to deploy a model, it's critical that you preserve the preprocessing calculations. The easiest way to implement them as layers, and attach them to your model before export.


### Look at the data distribution

Next compare the distributions of the positive and negative examples over a few features. Good questions to ask yourself at this point are:

* Do these distributions make sense? 
    * Yes. You've normalized the input and these are mostly concentrated in the `+/- 2` range.
* Can you see the difference between the distributions?
    * Yes the positive examples contain a much higher rate of extreme values.

In [11]:
pos_df = pd.DataFrame(train_features[ bool_train_labels], columns=train_df.columns)
neg_df = pd.DataFrame(train_features[~bool_train_labels], columns=train_df.columns)

sns.jointplot(x=pos_df['V5'], y=pos_df['V6'],
              kind='hex', xlim=(-5,5), ylim=(-5,5))
plt.suptitle("Positive distribution")

sns.jointplot(x=neg_df['V5'], y=neg_df['V6'],
              kind='hex', xlim=(-5,5), ylim=(-5,5))
_ = plt.suptitle("Negative distribution")

## Define the model and metrics

Define a function that creates a simple neural network with a densly connected hidden layer, a [dropout](https://developers.google.com/machine-learning/glossary/#dropout_regularization) layer to reduce overfitting, and an output sigmoid layer that returns the probability of a transaction being fraudulent: 

In [12]:
METRICS = [
      keras.metrics.BinaryCrossentropy(name='cross entropy'),  # same as model's loss
      keras.metrics.MeanSquaredError(name='Brier score'),
      keras.metrics.TruePositives(name='tp'),
      keras.metrics.FalsePositives(name='fp'),
      keras.metrics.TrueNegatives(name='tn'),
      keras.metrics.FalseNegatives(name='fn'), 
      keras.metrics.BinaryAccuracy(name='accuracy'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(name='auc'),
      keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
]

def make_model(metrics=METRICS, output_bias=None):
  if output_bias is not None:
    output_bias = tf.keras.initializers.Constant(output_bias)
  model = keras.Sequential([
      keras.layers.Dense(
          16, activation='relu',
          input_shape=(train_features.shape[-1],)),
      keras.layers.Dropout(0.5),
      keras.layers.Dense(1, activation='sigmoid',
                         bias_initializer=output_bias),
  ])

  model.compile(
      optimizer=keras.optimizers.Adam(learning_rate=1e-3),
      loss=keras.losses.BinaryCrossentropy(),
      metrics=metrics)

  return model

### Understanding useful metrics

Notice that there are a few metrics defined above that can be computed by the model that will be helpful when evaluating the performance.
These can be divided into three groups.

#### Metrics for probability predictions

As we train our network with the cross entropy as a loss function, it is fully capable of predicting class probabilities, i.e. it is a probabilistic classifier.
Good metrics to assess probabilistic predictions are, in fact, **proper scoring rules**. Their key property is that predicting the true probability is optimal. We give two well-known examples:

*   **cross entropy** also known as log loss
*   **Mean squared error** also known as the Brier score

#### Metrics for deterministic 0/1 predictions

In the end, one often wants to predict a class label, 0 or 1, *no fraud* or *fraud*.
This is called a deterministic classifier.
To get a label prediction from our probabilistic classifier, one needs to choose a probability threshold $t$.
The default is to predict label 1 (fraud) if the predicted probability is larger than $t=50\%$ and all the following metrics implicitly use this default. 

*   **False** negatives and **false** positives are samples that were **incorrectly** classified
*   **True** negatives and **true** positives are samples that were **correctly** classified
*   **Accuracy** is the percentage of examples correctly classified
>   $\frac{\text{true samples}}{\text{total samples}}$
*   **Precision** is the percentage of **predicted** positives that were correctly classified
>   $\frac{\text{true positives}}{\text{true positives + false positives}}$
*   **Recall** is the percentage of **actual** positives that were correctly classified
>   $\frac{\text{true positives}}{\text{true positives + false negatives}}$

**Note:** Accuracy is not a helpful metric for this task. You can have 99.8%+ accuracy on this task by predicting False all the time.  

#### Other metrices

The following metrics take into account all possible choices of thresholds $t$.

*   **AUC** refers to the Area Under the Curve of a Receiver Operating Characteristic curve (ROC-AUC). This metric is equal to the probability that a classifier will rank a random positive sample higher than a random negative sample.
*   **AUPRC** refers to Area Under the Curve of the Precision-Recall Curve. This metric computes precision-recall pairs for different probability thresholds. 


#### Read more:
*   [Strictly Proper Scoring Rules, Prediction, and Estimation](https://www.stat.washington.edu/people/raftery/Research/PDF/Gneiting2007jasa.pdf)
*   [True vs. False and Positive vs. Negative](https://developers.google.com/machine-learning/crash-course/classification/true-false-positive-negative)
*   [Accuracy](https://developers.google.com/machine-learning/crash-course/classification/accuracy)
*   [Precision and Recall](https://developers.google.com/machine-learning/crash-course/classification/precision-and-recall)
*   [ROC-AUC](https://developers.google.com/machine-learning/crash-course/classification/roc-and-auc)
*   [Relationship between Precision-Recall and ROC Curves](https://www.biostat.wisc.edu/~page/rocpr.pdf)

## Baseline model

### Build the model

Now create and train your model using the function that was defined earlier. Notice that the model is fit using a larger than default batch size of 2048, this is important to ensure that each batch has a decent chance of containing a few positive samples. If the batch size was too small, they would likely have no fraudulent transactions to learn from.


Note: Fitting this model will not handle the class imbalance efficiently. You will improve it later in this tutorial.

In [13]:
EPOCHS = 100
BATCH_SIZE = 2048

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_prc', 
    verbose=1,
    patience=10,
    mode='max',
    restore_best_weights=True)

In [14]:
model = make_model()
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 16)                480       
                                                                 
 dropout (Dropout)           (None, 16)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 17        
                                                                 
Total params: 497
Trainable params: 497
Non-trainable params: 0
_________________________________________________________________


Test run the model:

In [15]:
model.predict(train_features[:10])

array([[0.7681104 ],
       [0.90654016],
       [0.6341982 ],
       [0.1439878 ],
       [0.3255878 ],
       [0.86907613],
       [0.55202377],
       [0.9646705 ],
       [0.7009744 ],
       [0.66902274]], dtype=float32)

### Optional: Set the correct initial bias.

These initial guesses are not great. You know the dataset is imbalanced. Set the output layer's bias to reflect that, see [A Recipe for Training Neural Networks: "init well"](http://karpathy.github.io/2019/04/25/recipe/#2-set-up-the-end-to-end-trainingevaluation-skeleton--get-dumb-baselines). This can help with initial convergence.

With the default bias initialization the loss should be about `math.log(2) = 0.69314` 

In [16]:
results = model.evaluate(train_features, train_labels, batch_size=BATCH_SIZE, verbose=0)
print("Loss: {:0.4f}".format(results[0]))

Loss: 1.3599


The correct bias to set can be derived from:

$$ p_0 = pos/(pos + neg) = 1/(1+e^{-b_0}) $$
$$ b_0 = -log_e(1/p_0 - 1) $$
$$ b_0 = log_e(pos/neg)$$

In [17]:
initial_bias = np.log([pos/neg])
initial_bias

array([-6.35935934])

Set that as the initial bias, and the model will give much more reasonable initial guesses. 

It should be near: `pos/total = 0.0018`

In [18]:
model = make_model(output_bias=initial_bias)
model.predict(train_features[:10])

array([[0.00166613],
       [0.00043574],
       [0.00146988],
       [0.00160271],
       [0.00081715],
       [0.00049797],
       [0.00059584],
       [0.00119504],
       [0.00135604],
       [0.00100744]], dtype=float32)

With this initialization the initial loss should be approximately:

$$-p_0log(p_0)-(1-p_0)log(1-p_0) = 0.01317$$

In [19]:
results = model.evaluate(train_features, train_labels, batch_size=BATCH_SIZE, verbose=0)
print("Loss: {:0.4f}".format(results[0]))

Loss: 0.0150


This initial loss is about 50 times less than it would have been with naive initialization.

This way the model doesn't need to spend the first few epochs just learning that positive examples are unlikely. It also makes it easier to read plots of the loss during training.

### Checkpoint the initial weights

To make the various training runs more comparable, keep this initial model's weights in a checkpoint file, and load them into each model before training:

In [20]:
initial_weights = os.path.join(tempfile.mkdtemp(), 'initial_weights')
model.save_weights(initial_weights)

### Confirm that the bias fix helps

Before moving on, confirm quick that the careful bias initialization actually helped.

Train the model for 20 epochs, with and without this careful initialization, and compare the losses: 

In [21]:
model = make_model()
model.load_weights(initial_weights)
model.layers[-1].bias.assign([0.0])
zero_bias_history = model.fit(
    train_features,
    train_labels,
    batch_size=BATCH_SIZE,
    epochs=20,
    validation_data=(val_features, val_labels), 
    verbose=0)

In [22]:
model = make_model()
model.load_weights(initial_weights)
careful_bias_history = model.fit(
    train_features,
    train_labels,
    batch_size=BATCH_SIZE,
    epochs=20,
    validation_data=(val_features, val_labels), 
    verbose=0)

In [23]:
def plot_loss(history, label, n):
  # Use a log scale on y-axis to show the wide range of values.
  plt.semilogy(history.epoch, history.history['loss'],
               color=colors[n], label='Train ' + label)
  plt.semilogy(history.epoch, history.history['val_loss'],
               color=colors[n], label='Val ' + label,
               linestyle="--")
  plt.xlabel('Epoch')
  plt.ylabel('Loss')
  plt.legend()

In [24]:
plot_loss(zero_bias_history, "Zero Bias", 0)
plot_loss(careful_bias_history, "Careful Bias", 1)

The above figure makes it clear: In terms of validation loss, on this problem, this careful initialization gives a clear advantage. 

### Train the model

In [25]:
model = make_model()
model.load_weights(initial_weights)
baseline_history = model.fit(
    train_features,
    train_labels,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    callbacks=[early_stopping],
    validation_data=(val_features, val_labels))

Epoch 1/100
90/90 [==============================] - 8s 22ms/step - loss: 0.0116 - cross entropy: 0.0098 - Brier score: 0.0014 - tp: 79.0000 - fp: 23.0000 - tn: 227427.0000 - fn: 316.0000 - accuracy: 0.9985 - precision: 0.7745 - recall: 0.2000 - auc: 0.7652 - prc: 0.2853 - val_loss: 0.0050 - val_cross entropy: 0.0050 - val_Brier score: 9.7668e-04 - val_tp: 13.0000 - val_fp: 2.0000 - val_tn: 45501.0000 - val_fn: 53.0000 - val_accuracy: 0.9988 - val_precision: 0.8667 - val_recall: 0.1970 - val_auc: 0.9238 - val_prc: 0.7091
Epoch 2/100
90/90 [==============================] - 0s 5ms/step - loss: 0.0080 - cross entropy: 0.0080 - Brier score: 0.0013 - tp: 88.0000 - fp: 26.0000 - tn: 181921.0000 - fn: 241.0000 - accuracy: 0.9985 - precision: 0.7719 - recall: 0.2675 - auc: 0.8514 - prc: 0.4406 - val_loss: 0.0036 - val_cross entropy: 0.0036 - val_Brier score: 6.5637e-04 - val_tp: 34.0000 - val_fp: 5.0000 - val_tn: 45498.0000 - val_fn: 32.0000 - val_accuracy: 0.9992 - val_precision: 0.8718 - va

90/90 [==============================] - 0s 6ms/step - loss: 0.0049 - cross entropy: 0.0049 - Brier score: 8.6313e-04 - tp: 181.0000 - fp: 39.0000 - tn: 181908.0000 - fn: 148.0000 - accuracy: 0.9990 - precision: 0.8227 - recall: 0.5502 - auc: 0.9076 - prc: 0.6815 - val_loss: 0.0023 - val_cross entropy: 0.0023 - val_Brier score: 4.0476e-04 - val_tp: 54.0000 - val_fp: 10.0000 - val_tn: 45493.0000 - val_fn: 12.0000 - val_accuracy: 0.9995 - val_precision: 0.8438 - val_recall: 0.8182 - val_auc: 0.9543 - val_prc: 0.8333
Epoch 17/100
90/90 [==============================] - 0s 5ms/step - loss: 0.0048 - cross entropy: 0.0048 - Brier score: 8.0107e-04 - tp: 192.0000 - fp: 23.0000 - tn: 181924.0000 - fn: 137.0000 - accuracy: 0.9991 - precision: 0.8930 - recall: 0.5836 - auc: 0.9182 - prc: 0.6857 - val_loss: 0.0022 - val_cross entropy: 0.0022 - val_Brier score: 4.0278e-04 - val_tp: 54.0000 - val_fp: 10.0000 - val_tn: 45493.0000 - val_fn: 12.0000 - val_accuracy: 0.9995 - val_precision: 0.8438 - va

90/90 [==============================] - 0s 4ms/step - loss: 0.0043 - cross entropy: 0.0043 - Brier score: 7.9374e-04 - tp: 195.0000 - fp: 31.0000 - tn: 181916.0000 - fn: 134.0000 - accuracy: 0.9991 - precision: 0.8628 - recall: 0.5927 - auc: 0.9260 - prc: 0.7095 - val_loss: 0.0021 - val_cross entropy: 0.0021 - val_Brier score: 3.9535e-04 - val_tp: 53.0000 - val_fp: 8.0000 - val_tn: 45495.0000 - val_fn: 13.0000 - val_accuracy: 0.9995 - val_precision: 0.8689 - val_recall: 0.8030 - val_auc: 0.9543 - val_prc: 0.8515
Epoch 32/100
90/90 [==============================] - 0s 5ms/step - loss: 0.0042 - cross entropy: 0.0042 - Brier score: 8.0218e-04 - tp: 185.0000 - fp: 30.0000 - tn: 181917.0000 - fn: 144.0000 - accuracy: 0.9990 - precision: 0.8605 - recall: 0.5623 - auc: 0.9260 - prc: 0.7066 - val_loss: 0.0022 - val_cross entropy: 0.0022 - val_Brier score: 3.9453e-04 - val_tp: 56.0000 - val_fp: 9.0000 - val_tn: 45494.0000 - val_fn: 10.0000 - val_accuracy: 0.9996 - val_precision: 0.8615 - val_

90/90 [==============================] - 0s 4ms/step - loss: 0.0039 - cross entropy: 0.0039 - Brier score: 7.4382e-04 - tp: 205.0000 - fp: 38.0000 - tn: 181909.0000 - fn: 124.0000 - accuracy: 0.9991 - precision: 0.8436 - recall: 0.6231 - auc: 0.9427 - prc: 0.7456 - val_loss: 0.0021 - val_cross entropy: 0.0021 - val_Brier score: 3.9885e-04 - val_tp: 53.0000 - val_fp: 10.0000 - val_tn: 45493.0000 - val_fn: 13.0000 - val_accuracy: 0.9995 - val_precision: 0.8413 - val_recall: 0.8030 - val_auc: 0.9543 - val_prc: 0.8572
Epoch 47/100
90/90 [==============================] - 0s 4ms/step - loss: 0.0039 - cross entropy: 0.0039 - Brier score: 7.6381e-04 - tp: 197.0000 - fp: 26.0000 - tn: 181921.0000 - fn: 132.0000 - accuracy: 0.9991 - precision: 0.8834 - recall: 0.5988 - auc: 0.9336 - prc: 0.7461 - val_loss: 0.0021 - val_cross entropy: 0.0021 - val_Brier score: 3.9683e-04 - val_tp: 56.0000 - val_fp: 10.0000 - val_tn: 45493.0000 - val_fn: 10.0000 - val_accuracy: 0.9996 - val_precision: 0.8485 - va

90/90 [==============================] - 0s 5ms/step - loss: 0.0036 - cross entropy: 0.0036 - Brier score: 6.7965e-04 - tp: 212.0000 - fp: 28.0000 - tn: 181919.0000 - fn: 117.0000 - accuracy: 0.9992 - precision: 0.8833 - recall: 0.6444 - auc: 0.9321 - prc: 0.7600 - val_loss: 0.0021 - val_cross entropy: 0.0021 - val_Brier score: 3.9229e-04 - val_tp: 56.0000 - val_fp: 10.0000 - val_tn: 45493.0000 - val_fn: 10.0000 - val_accuracy: 0.9996 - val_precision: 0.8485 - val_recall: 0.8485 - val_auc: 0.9543 - val_prc: 0.8582
Epoch 61: early stopping


### Check training history

In this section, you will produce plots of your model's accuracy and loss on the training and validation set. These are useful to check for overfitting, which you can learn more about in the [Overfit and underfit](https://www.tensorflow.org/tutorials/keras/overfit_and_underfit) tutorial.

Additionally, you can produce these plots for any of the metrics you created above. False negatives are included as an example.

In [26]:
def plot_metrics(history):
  metrics = ['loss', 'prc', 'precision', 'recall']
  for n, metric in enumerate(metrics):
    name = metric.replace("_"," ").capitalize()
    plt.subplot(2,2,n+1)
    plt.plot(history.epoch, history.history[metric], color=colors[0], label='Train')
    plt.plot(history.epoch, history.history['val_'+metric],
             color=colors[0], linestyle="--", label='Val')
    plt.xlabel('Epoch')
    plt.ylabel(name)
    if metric == 'loss':
      plt.ylim([0, plt.ylim()[1]])
    elif metric == 'auc':
      plt.ylim([0.8,1])
    else:
      plt.ylim([0,1])

    plt.legend()

In [27]:
plot_metrics(baseline_history)

Note: That the validation curve generally performs better than the training curve. This is mainly caused by the fact that the dropout layer is not active when evaluating the model.

### Evaluate metrics

You can use a [confusion matrix](https://developers.google.com/machine-learning/glossary/#confusion_matrix) to summarize the actual vs. predicted labels, where the X axis is the predicted label and the Y axis is the actual label:

In [28]:
train_predictions_baseline = model.predict(train_features, batch_size=BATCH_SIZE)
test_predictions_baseline = model.predict(test_features, batch_size=BATCH_SIZE)

In [29]:
def plot_cm(labels, predictions, threshold=0.5):
  cm = confusion_matrix(labels, predictions > threshold)
  plt.figure(figsize=(5,5))
  sns.heatmap(cm, annot=True, fmt="d")
  plt.title('Confusion matrix @{:.2f}'.format(threshold))
  plt.ylabel('Actual label')
  plt.xlabel('Predicted label')

  print('Legitimate Transactions Detected (True Negatives): ', cm[0][0])
  print('Legitimate Transactions Incorrectly Detected (False Positives): ', cm[0][1])
  print('Fraudulent Transactions Missed (False Negatives): ', cm[1][0])
  print('Fraudulent Transactions Detected (True Positives): ', cm[1][1])
  print('Total Fraudulent Transactions: ', np.sum(cm[1]))

Evaluate your model on the test dataset and display the results for the metrics you created above:

In [30]:
baseline_results = model.evaluate(test_features, test_labels,
                                  batch_size=BATCH_SIZE, verbose=0)
for name, value in zip(model.metrics_names, baseline_results):
  print(name, ': ', value)
print()

plot_cm(test_labels, test_predictions_baseline)

loss :  0.002692745765671134
cross entropy :  0.002692745765671134
Brier score :  0.0004214174987282604
tp :  77.0
fp :  7.0
tn :  56858.0
fn :  20.0
accuracy :  0.9995260238647461
precision :  0.9166666865348816
recall :  0.7938144207000732
auc :  0.9429294466972351
prc :  0.8425734639167786

Legitimate Transactions Detected (True Negatives):  56858
Legitimate Transactions Incorrectly Detected (False Positives):  7
Fraudulent Transactions Missed (False Negatives):  20
Fraudulent Transactions Detected (True Positives):  77
Total Fraudulent Transactions:  97


If the model had predicted everything perfectly (impossible with true randomness), this would be a [diagonal matrix](https://en.wikipedia.org/wiki/Diagonal_matrix) where values off the main diagonal, indicating incorrect predictions, would be zero. In this case, the matrix shows that you have relatively few false positives, meaning that there were relatively few legitimate transactions that were incorrectly flagged.

### Changing the threshold

The default threshold of $t=50\%$ corresponds to equal costs of false negatives and false positives.
In the case of fraud detection, however, you would likely associate higher costs to false negatives than to false positives.
This trade off may be preferable because false negatives would allow fraudulent transactions to go through, whereas false positives may cause an email to be sent to a customer to ask them to verify their card activity.

By decreasing the threshold, we attribute higher cost to false negatives, thereby increasing missed transactions at the price of more false positives.
We test thresholds at 10% and at 1%.

In [31]:
plot_cm(test_labels, test_predictions_baseline, threshold=0.1)
plot_cm(test_labels, test_predictions_baseline, threshold=0.01)

Legitimate Transactions Detected (True Negatives):  56853
Legitimate Transactions Incorrectly Detected (False Positives):  12
Fraudulent Transactions Missed (False Negatives):  16
Fraudulent Transactions Detected (True Positives):  81
Total Fraudulent Transactions:  97
Legitimate Transactions Detected (True Negatives):  56772
Legitimate Transactions Incorrectly Detected (False Positives):  93
Fraudulent Transactions Missed (False Negatives):  15
Fraudulent Transactions Detected (True Positives):  82
Total Fraudulent Transactions:  97


### Plot the ROC

Now plot the [ROC](https://developers.google.com/machine-learning/glossary#ROC). This plot is useful because it shows, at a glance, the range of performance the model can reach by tuning the output threshold over its full range (0 to 1). So each point corresponds to a single value of the threshold.

In [32]:
def plot_roc(name, labels, predictions, **kwargs):
  fp, tp, _ = sklearn.metrics.roc_curve(labels, predictions)

  plt.plot(100*fp, 100*tp, label=name, linewidth=2, **kwargs)
  plt.xlabel('False positives [%]')
  plt.ylabel('True positives [%]')
  plt.xlim([-0.5,20])
  plt.ylim([80,100.5])
  plt.grid(True)
  ax = plt.gca()
  ax.set_aspect('equal')

In [33]:
plot_roc("Train Baseline", train_labels, train_predictions_baseline, color=colors[0])
plot_roc("Test Baseline", test_labels, test_predictions_baseline, color=colors[0], linestyle='--')
plt.legend(loc='lower right');

### Plot the PRC

Now plot the [AUPRC](https://developers.google.com/machine-learning/glossary?hl=en#PR_AUC). Area under the interpolated precision-recall curve, obtained by plotting (recall, precision) points for different values of the classification threshold. Depending on how it's calculated, PR AUC may be equivalent to the average precision of the model.


In [34]:
def plot_prc(name, labels, predictions, **kwargs):
    precision, recall, _ = sklearn.metrics.precision_recall_curve(labels, predictions)

    plt.plot(precision, recall, label=name, linewidth=2, **kwargs)
    plt.xlabel('Precision')
    plt.ylabel('Recall')
    plt.grid(True)
    ax = plt.gca()
    ax.set_aspect('equal')

In [35]:
plot_prc("Train Baseline", train_labels, train_predictions_baseline, color=colors[0])
plot_prc("Test Baseline", test_labels, test_predictions_baseline, color=colors[0], linestyle='--')
plt.legend(loc='lower right');

It looks like the precision is relatively high, but the recall and the area under the ROC curve (AUC) aren't as high as you might like. Classifiers often face challenges when trying to maximize both precision and recall, which is especially true when working with imbalanced datasets. It is important to consider the costs of different types of errors in the context of the problem you care about. In this example, a false negative (a fraudulent transaction is missed) may have a financial cost, while a false positive (a transaction is incorrectly flagged as fraudulent) may decrease user happiness.

## Class weights

### Calculate class weights

The goal is to identify fraudulent transactions, but you don't have very many of those positive samples to work with, so you would want to have the classifier heavily weight the few examples that are available. You can do this by passing Keras weights for each class through a parameter. These will cause the model to "pay more attention" to examples from an under-represented class. Note, however, that this does not increase in any way the amount of information of your dataset. In the end, using class weights is more or less equivalent to changing the output bias or to changing the threshold. Let's see how it works out.

In [36]:
# Scaling by total/2 helps keep the loss to a similar magnitude.
# The sum of the weights of all examples stays the same.
weight_for_0 = (1 / neg) * (total / 2.0)
weight_for_1 = (1 / pos) * (total / 2.0)

class_weight = {0: weight_for_0, 1: weight_for_1}

print('Weight for class 0: {:.2f}'.format(weight_for_0))
print('Weight for class 1: {:.2f}'.format(weight_for_1))

Weight for class 0: 0.50
Weight for class 1: 289.44


### Train a model with class weights

Now try re-training and evaluating the model with class weights to see how that affects the predictions.

Note: Using `class_weights` changes the range of the loss. This may affect the stability of the training depending on the optimizer. Optimizers whose step size is dependent on the magnitude of the gradient, like `tf.keras.optimizers.SGD`, may fail. The optimizer used here, `tf.keras.optimizers.Adam`, is unaffected by the scaling change. Also note that because of the weighting, the total losses are not comparable between the two models.

In [37]:
weighted_model = make_model()
weighted_model.load_weights(initial_weights)

weighted_history = weighted_model.fit(
    train_features,
    train_labels,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    callbacks=[early_stopping],
    validation_data=(val_features, val_labels),
    # The class weights go here
    class_weight=class_weight) 

Epoch 1/100
90/90 [==============================] - 5s 23ms/step - loss: 2.5327 - cross entropy: 0.0096 - Brier score: 0.0014 - tp: 115.0000 - fp: 72.0000 - tn: 238740.0000 - fn: 311.0000 - accuracy: 0.9984 - precision: 0.6150 - recall: 0.2700 - auc: 0.7867 - prc: 0.3237 - val_loss: 0.0052 - val_cross entropy: 0.0052 - val_Brier score: 8.4343e-04 - val_tp: 24.0000 - val_fp: 5.0000 - val_tn: 45498.0000 - val_fn: 42.0000 - val_accuracy: 0.9990 - val_precision: 0.8276 - val_recall: 0.3636 - val_auc: 0.9277 - val_prc: 0.6120
Epoch 2/100
90/90 [==============================] - 0s 5ms/step - loss: 1.3374 - cross entropy: 0.0111 - Brier score: 0.0017 - tp: 148.0000 - fp: 177.0000 - tn: 181770.0000 - fn: 181.0000 - accuracy: 0.9980 - precision: 0.4554 - recall: 0.4498 - auc: 0.8603 - prc: 0.3985 - val_loss: 0.0057 - val_cross entropy: 0.0057 - val_Brier score: 6.6268e-04 - val_tp: 50.0000 - val_fp: 16.0000 - val_tn: 45487.0000 - val_fn: 16.0000 - val_accuracy: 0.9993 - val_precision: 0.7576 

Epoch 15: early stopping


### Check training history

In [38]:
plot_metrics(weighted_history)

### Evaluate metrics

In [39]:
train_predictions_weighted = weighted_model.predict(train_features, batch_size=BATCH_SIZE)
test_predictions_weighted = weighted_model.predict(test_features, batch_size=BATCH_SIZE)

In [40]:
weighted_results = weighted_model.evaluate(test_features, test_labels,
                                           batch_size=BATCH_SIZE, verbose=0)
for name, value in zip(weighted_model.metrics_names, weighted_results):
  print(name, ': ', value)
print()

plot_cm(test_labels, test_predictions_weighted)

loss :  0.015621509402990341
cross entropy :  0.015621509402990341
Brier score :  0.002418939722701907
tp :  83.0
fp :  139.0
tn :  56726.0
fn :  14.0
accuracy :  0.9973139762878418
precision :  0.37387385964393616
recall :  0.8556700944900513
auc :  0.9491804838180542
prc :  0.7415635585784912

Legitimate Transactions Detected (True Negatives):  56726
Legitimate Transactions Incorrectly Detected (False Positives):  139
Fraudulent Transactions Missed (False Negatives):  14
Fraudulent Transactions Detected (True Positives):  83
Total Fraudulent Transactions:  97


Here you can see that with class weights the accuracy and precision are lower because there are more false positives, but conversely the recall and AUC are higher because the model also found more true positives. Despite having lower accuracy, this model has higher recall (and identifies more fraudulent transactions than the baseline model at threshold 50%). Of course, there is a cost to both types of error (you wouldn't want to bug users by flagging too many legitimate transactions as fraudulent, either). Carefully consider the trade-offs between these different types of errors for your application.

Compared to the baseline model with changed threshold, the class weighted model is clearly inferior. The superiority of the baseline model is further confirmed by the lower test loss value (cross entropy and mean squared error) and additionally can be seen by plotting the ROC curves of both models together.

### Plot the ROC

In [41]:
plot_roc("Train Baseline", train_labels, train_predictions_baseline, color=colors[0])
plot_roc("Test Baseline", test_labels, test_predictions_baseline, color=colors[0], linestyle='--')

plot_roc("Train Weighted", train_labels, train_predictions_weighted, color=colors[1])
plot_roc("Test Weighted", test_labels, test_predictions_weighted, color=colors[1], linestyle='--')


plt.legend(loc='lower right');

### Plot the PRC

In [42]:
plot_prc("Train Baseline", train_labels, train_predictions_baseline, color=colors[0])
plot_prc("Test Baseline", test_labels, test_predictions_baseline, color=colors[0], linestyle='--')

plot_prc("Train Weighted", train_labels, train_predictions_weighted, color=colors[1])
plot_prc("Test Weighted", test_labels, test_predictions_weighted, color=colors[1], linestyle='--')


plt.legend(loc='lower right');

## Oversampling

### Oversample the minority class

A related approach would be to resample the dataset by oversampling the minority class.

In [43]:
pos_features = train_features[bool_train_labels]
neg_features = train_features[~bool_train_labels]

pos_labels = train_labels[bool_train_labels]
neg_labels = train_labels[~bool_train_labels]

#### Using NumPy

You can balance the dataset manually by choosing the right number of random 
indices from the positive examples:

In [44]:
ids = np.arange(len(pos_features))
choices = np.random.choice(ids, len(neg_features))

res_pos_features = pos_features[choices]
res_pos_labels = pos_labels[choices]

res_pos_features.shape

(181947, 29)

In [45]:
resampled_features = np.concatenate([res_pos_features, neg_features], axis=0)
resampled_labels = np.concatenate([res_pos_labels, neg_labels], axis=0)

order = np.arange(len(resampled_labels))
np.random.shuffle(order)
resampled_features = resampled_features[order]
resampled_labels = resampled_labels[order]

resampled_features.shape

(363894, 29)

#### Using `tf.data`

If you're using `tf.data` the easiest way to produce balanced examples is to start with a `positive` and a `negative` dataset, and merge them. See [the tf.data guide](../../guide/data.ipynb) for more examples.

In [46]:
BUFFER_SIZE = 100000

def make_ds(features, labels):
  ds = tf.data.Dataset.from_tensor_slices((features, labels))#.cache()
  ds = ds.shuffle(BUFFER_SIZE).repeat()
  return ds

pos_ds = make_ds(pos_features, pos_labels)
neg_ds = make_ds(neg_features, neg_labels)

Each dataset provides `(feature, label)` pairs:

In [47]:
for features, label in pos_ds.take(1):
  print("Features:\n", features.numpy())
  print()
  print("Label: ", label.numpy())

Features:
 [-3.78197692 -4.51862832 -3.10537408  2.62774362  4.5380158  -4.83524305
 -4.77803628  1.21810765 -0.5874832  -3.71476199  2.47461248 -3.29819384
 -1.99791605 -5.         -0.9572825  -1.76620561 -2.2403598   0.59439953
  0.16838202  3.82008469  1.60532133 -1.34609337 -0.44933093 -1.04829141
  0.23582722  0.83973373  1.77561726 -3.77978759  0.30506335]

Label:  1


Merge the two together using `tf.data.Dataset.sample_from_datasets`:

In [48]:
resampled_ds = tf.data.Dataset.sample_from_datasets([pos_ds, neg_ds], weights=[0.5, 0.5])
resampled_ds = resampled_ds.batch(BATCH_SIZE).prefetch(2)

In [49]:
for features, label in resampled_ds.take(1):
  print(label.numpy().mean())

0.5029296875


To use this dataset, you'll need the number of steps per epoch.

The definition of "epoch" in this case is less clear. Say it's the number of batches required to see each negative example once:

In [50]:
resampled_steps_per_epoch = np.ceil(2.0*neg/BATCH_SIZE)
resampled_steps_per_epoch

278.0

### Train on the oversampled data

Now try training the model with the resampled data set instead of using class weights to see how these methods compare.

Note: Because the data was balanced by replicating the positive examples, the total dataset size is larger, and each epoch runs for more training steps. 

In [51]:
resampled_model = make_model()
resampled_model.load_weights(initial_weights)

# Reset the bias to zero, since this dataset is balanced.
output_layer = resampled_model.layers[-1] 
output_layer.bias.assign([0])

val_ds = tf.data.Dataset.from_tensor_slices((val_features, val_labels)).cache()
val_ds = val_ds.batch(BATCH_SIZE).prefetch(2) 

resampled_history = resampled_model.fit(
    resampled_ds,
    epochs=EPOCHS,
    steps_per_epoch=resampled_steps_per_epoch,
    callbacks=[early_stopping],
    validation_data=val_ds)

Epoch 1/100
278/278 [==============================] - 17s 36ms/step - loss: 0.4211 - cross entropy: 0.3842 - Brier score: 0.1193 - tp: 220814.0000 - fp: 42429.0000 - tn: 299241.0000 - fn: 63822.0000 - accuracy: 0.8304 - precision: 0.8388 - recall: 0.7758 - auc: 0.9013 - prc: 0.9137 - val_loss: 0.1594 - val_cross entropy: 0.1594 - val_Brier score: 0.0320 - val_tp: 60.0000 - val_fp: 894.0000 - val_tn: 44609.0000 - val_fn: 6.0000 - val_accuracy: 0.9802 - val_precision: 0.0629 - val_recall: 0.9091 - val_auc: 0.9770 - val_prc: 0.7710
Epoch 2/100
278/278 [==============================] - 7s 25ms/step - loss: 0.2066 - cross entropy: 0.2066 - Brier score: 0.0628 - tp: 245583.0000 - fp: 14190.0000 - tn: 270815.0000 - fn: 38756.0000 - accuracy: 0.9070 - precision: 0.9454 - recall: 0.8637 - auc: 0.9720 - prc: 0.9767 - val_loss: 0.0902 - val_cross entropy: 0.0902 - val_Brier score: 0.0176 - val_tp: 60.0000 - val_fp: 677.0000 - val_tn: 44826.0000 - val_fn: 6.0000 - val_accuracy: 0.9850 - val_prec

If the training process were considering the whole dataset on each gradient update, this oversampling would be basically identical to the class weighting.

But when training the model batch-wise, as you did here, the oversampled data provides a smoother gradient signal: Instead of each positive example being shown in one batch with a large weight, they're shown in many different batches each time with a small weight. 

This smoother gradient signal makes it easier to train the model.

### Check training history

Note that the distributions of metrics will be different here, because the training data has a totally different distribution from the validation and test data. 

In [52]:
plot_metrics(resampled_history)

### Re-train


Because training is easier on the balanced data, the above training procedure may overfit quickly. 

So break up the epochs to give the `tf.keras.callbacks.EarlyStopping` finer control over when to stop training.

In [53]:
resampled_model = make_model()
resampled_model.load_weights(initial_weights)

# Reset the bias to zero, since this dataset is balanced.
output_layer = resampled_model.layers[-1] 
output_layer.bias.assign([0])

resampled_history = resampled_model.fit(
    resampled_ds,
    # These are not real epochs
    steps_per_epoch=20,
    epochs=10*EPOCHS,
    callbacks=[early_stopping],
    validation_data=(val_ds))

Epoch 1/1000
20/20 [==============================] - 6s 133ms/step - loss: 0.9597 - cross entropy: 0.4700 - Brier score: 0.1411 - tp: 9279.0000 - fp: 5594.0000 - tn: 60496.0000 - fn: 11160.0000 - accuracy: 0.8064 - precision: 0.6239 - recall: 0.4540 - auc: 0.8413 - prc: 0.6372 - val_loss: 0.4542 - val_cross entropy: 0.4542 - val_Brier score: 0.1454 - val_tp: 53.0000 - val_fp: 9078.0000 - val_tn: 36425.0000 - val_fn: 13.0000 - val_accuracy: 0.8005 - val_precision: 0.0058 - val_recall: 0.8030 - val_auc: 0.8701 - val_prc: 0.0624
Epoch 2/1000
20/20 [==============================] - 1s 37ms/step - loss: 0.6391 - cross entropy: 0.6391 - Brier score: 0.2026 - tp: 14073.0000 - fp: 5361.0000 - tn: 15142.0000 - fn: 6384.0000 - accuracy: 0.7133 - precision: 0.7241 - recall: 0.6879 - auc: 0.7602 - prc: 0.8184 - val_loss: 0.4493 - val_cross entropy: 0.4493 - val_Brier score: 0.1426 - val_tp: 59.0000 - val_fp: 8476.0000 - val_tn: 37027.0000 - val_fn: 7.0000 - val_accuracy: 0.8138 - val_precision: 

20/20 [==============================] - 1s 35ms/step - loss: 0.2349 - cross entropy: 0.2349 - Brier score: 0.0711 - tp: 17600.0000 - fp: 1203.0000 - tn: 19148.0000 - fn: 3009.0000 - accuracy: 0.8972 - precision: 0.9360 - recall: 0.8540 - auc: 0.9630 - prc: 0.9709 - val_loss: 0.1409 - val_cross entropy: 0.1409 - val_Brier score: 0.0275 - val_tp: 60.0000 - val_fp: 839.0000 - val_tn: 44664.0000 - val_fn: 6.0000 - val_accuracy: 0.9815 - val_precision: 0.0667 - val_recall: 0.9091 - val_auc: 0.9799 - val_prc: 0.7645
Epoch 17/1000
20/20 [==============================] - 1s 31ms/step - loss: 0.2266 - cross entropy: 0.2266 - Brier score: 0.0688 - tp: 17397.0000 - fp: 1182.0000 - tn: 19530.0000 - fn: 2851.0000 - accuracy: 0.9015 - precision: 0.9364 - recall: 0.8592 - auc: 0.9655 - prc: 0.9719 - val_loss: 0.1342 - val_cross entropy: 0.1342 - val_Brier score: 0.0261 - val_tp: 60.0000 - val_fp: 825.0000 - val_tn: 44678.0000 - val_fn: 6.0000 - val_accuracy: 0.9818 - val_precision: 0.0678 - val_rec

Epoch 30: early stopping


### Re-check training history

In [54]:
plot_metrics(resampled_history)

<ipython-input-26-d9248335d864>:5: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  plt.subplot(2,2,n+1)


### Evaluate metrics

In [55]:
train_predictions_resampled = resampled_model.predict(train_features, batch_size=BATCH_SIZE)
test_predictions_resampled = resampled_model.predict(test_features, batch_size=BATCH_SIZE)

In [56]:
train_features[0]

array([ 1.04758778,  0.07727343, -1.19163758,  0.15449483,  0.50939678,
       -0.21544273,  0.0763868 , -0.05938365,  0.16713332, -0.210343  ,
        0.96802119,  1.00661902,  0.5056653 , -0.78658977, -0.52064647,
        0.59231751,  0.16698334,  0.16424675,  0.47070943, -0.14539261,
       -0.43723178, -1.13516723,  0.46752395,  0.23104206, -0.47393609,
        0.36663968, -0.15312521, -0.13429526, -1.32857721])

In [57]:
resampled_results = resampled_model.evaluate(test_features, test_labels,
                                             batch_size=BATCH_SIZE, verbose=0)
for name, value in zip(resampled_model.metrics_names, resampled_results):
  print(name, ': ', value)
print()
plot_cm(test_labels, test_predictions_resampled)

loss :  0.1181606650352478
cross entropy :  0.1181606650352478
Brier score :  0.023037463426589966
tp :  87.0
fp :  1038.0
tn :  55827.0
fn :  10.0
accuracy :  0.9816017746925354
precision :  0.07733333110809326
recall :  0.8969072103500366
auc :  0.9815057516098022
prc :  0.7676520943641663

Legitimate Transactions Detected (True Negatives):  55827
Legitimate Transactions Incorrectly Detected (False Positives):  1038
Fraudulent Transactions Missed (False Negatives):  10
Fraudulent Transactions Detected (True Positives):  87
Total Fraudulent Transactions:  97


### Plot the ROC

In [58]:
plot_roc("Train Baseline", train_labels, train_predictions_baseline, color=colors[0])
plot_roc("Test Baseline", test_labels, test_predictions_baseline, color=colors[0], linestyle='--')
plot_roc("Train Weighted", train_labels, train_predictions_weighted, color=colors[1])
plot_roc("Test Weighted", test_labels, test_predictions_weighted, color=colors[1], linestyle='--')
plot_roc("Train Resampled", train_labels, train_predictions_resampled, color=colors[2])
plot_roc("Test Resampled", test_labels, test_predictions_resampled, color=colors[2], linestyle='--')
plt.legend(loc='lower right');

### Plot the AUPRC


In [59]:
plot_prc("Train Baseline", train_labels, train_predictions_baseline, color=colors[0])
plot_prc("Test Baseline", test_labels, test_predictions_baseline, color=colors[0], linestyle='--')

plot_prc("Train Weighted", train_labels, train_predictions_weighted, color=colors[1])
plot_prc("Test Weighted", test_labels, test_predictions_weighted, color=colors[1], linestyle='--')

plot_prc("Train Resampled", train_labels, train_predictions_resampled, color=colors[2])
plot_prc("Test Resampled", test_labels, test_predictions_resampled, color=colors[2], linestyle='--')
plt.legend(loc='lower right');

In [60]:
import tempfile

MODEL_DIR = "models"
version = 1
export_path = os.path.join(MODEL_DIR, str(version))
print('export_path = {}\n'.format(export_path))

tf.keras.models.save_model(
    model,
    export_path,
    overwrite=True,
    include_optimizer=True,
    save_format=None,
    signatures=None,
    options=None
)

print('\nSaved model:')
!ls -l {export_path}

export_path = models\1

INFO:tensorflow:Assets written to: models\1\assets

Saved model:


'ls' is not recognized as an internal or external command,
operable program or batch file.


## Prediction

In [72]:
import json
import requests

data = json.dumps({"signature_name": "serving_default", "instances": test_features[0:5].tolist()})
print('Data: {} ... {}'.format(data[:50], data[len(data)-52:]))


headers = {"content-type": "application/json"}
json_response = requests.post('http://34.72.253.181:80/v1/models/fraud_classifier:predict', data=data, headers=headers)
#json_response = requests.post('http://a723369bebfdc403da77ce94594d2bdf-939222270.us-east-1.elb.amazonaws.com/v1/models/fraud_classifier:predict', data=data, headers=headers)
print(json_response)
predictions = json.loads(json_response.text)['predictions']
print(predictions)

Data: {"signature_name": "serving_default", "instances": ... 05035318, 0.5593963793941588, -1.3285772065673767]]}
<Response [200]>
[[1.63317543e-06], [0.000104095539], [4.53663197e-05], [0.000200046663], [4.43129611e-05]]


Data: {"signature_name": "serving_default", "instances": ... 83953721, -2.056875217682629, -0.7236977124683363]]}
<Response [200]>
[[0.000217781009], [1.49573561e-05], [0.000214387343], [8.73693125e-07], [2.68517852e-07]]